# INITIALISATIONS

#CODE TO INITIALISE TENSORFLOW
from __future__ import division
import tensorflow as tf
from keras.backend.tensorflow_backend import set_session

#set_session(tf.Session(config=config))
print("Success!")

gpus = tf.config.experimental.list_physical_devices('GPU')
print(gpus)
for i in range(4):
    tf.config.experimental.set_virtual_device_configuration(
        gpus[i],
        tf.config.experimental.VirtualDeviceConfiguration(memory_limit=1024))


#Prints the devices in use. The next time I open this, should be configured to only use GPU 3
from tensorflow.python.client import device_lib
print(device_lib.list_local_devices())

In [ ]:
# Initializing TF and Keras

from __future__ import division
import sys
import tensorflow as tf
from keras.backend.tensorflow_backend import set_session
import keras

#FRANK# view versions
print("Tensorflow version is")
print(tf.__version__)
print("Keras version is")
print(keras.__version__)

#FRANK# these lines limiting memory usages are for tf2, which requires cudatoolkit>10 and newer driver
#gpus = tf.config.experimental.list_physical_devices('GPU')
#print(gpus)
#for i in range(4):   
#    tf.config.experimental.set_virtual_device_configuration(gpus[i],
#        [tf.config.experimental.VirtualDeviceConfiguration(memory_limit=5860500000)])

#config = tf.ConfigProto(log_device_placement=True)
#config.gpu_options.per_process_gpu_memory_fraction = 0.5
#sess = tf.Session(config=config)
#set_session(sess)
#print("Success!")

# Prints the devices in use. 
from tensorflow.python.client import device_lib
print(device_lib.list_local_devices())
print("Tensorflow version is")
print(tf.__version__)
print("Keras version is")
print(keras.__version__)

# Use only GPU3
tf.device('/device:XLA_GPU:3')

In [ ]:
# Importing Keras 
from keras.models import Sequential
from keras.models import Model
from keras.layers import Dense, Activation, Conv2D, MaxPooling2D, Flatten, Dropout
from keras.layers import Lambda
from keras import backend as K
from keras.engine.topology import Layer
from keras.layers.merge import concatenate
from keras.utils import plot_model
from keras.layers import SpatialDropout2D
from keras.preprocessing.image import ImageDataGenerator
from keras.callbacks import EarlyStopping
from keras.layers import Concatenate

# Import Libraries
import numpy as np
import math
import random
import os
import re
import matplotlib
import time
import pickle 

from itertools import groupby
from io import StringIO
from mpl_toolkits.axes_grid1 import make_axes_locatable

import pyjet
from pyjet import cluster, DTYPE_PTEPM
from pyjet.testdata import get_event

import matplotlib.pyplot as plt
import matplotlib.path as mpath
import matplotlib.lines as mlines
import matplotlib.patches as mpatches
from matplotlib.pyplot import cm
from matplotlib.colors import LinearSegmentedColormap
from matplotlib.collections import PatchCollection

import scipy
import scipy.optimize as opt
from scipy.interpolate import griddata
from scipy import interpolate
from scipy.optimize import least_squares

# Import local libraries
from substructure-variables.py import * # Jet substructure variables

In [ ]:
#IMPORTANT NOTE DON'T DELETE
#The 1e-3 factor in my weights comes from the fact that Pythia reports cross-sections in millibarns

#VARIABLES
width = 40
height = 40 #width, height of pictures

#FRANK# why do we need cross sec?
backgroundCross = 2.048e-06 #cross-section of processes in millibarns

actual_background_cross=2.84e-9 #barns
average_number_accepted=2162

actual_signal_cross = np.average([1.738e-14,1.7277e-14])
signal_accepted = np.average([8708-189,8827-172])
   

# READING IN DATA

In [ ]:
# This function reads off the original CSV format: 
# pt, eta, phi, m, id, isCharged \n
# pt, eta, phi, m, id, isCharged \n
# ... (of all particles in one event)
# trimmed mass of identified jet \n
# \n
# and produces a event_list
def return_event_list(fileName,max_Read = float("inf"),weighted=0,pt_cut=0):
    
    printed = 0
    
    event_list = [];mass_list = [];weight_list = [];
    tmp_events = open(fileName).read().split("\n\n")[:-1]
    print(len(tmp_events))
    for x in tmp_events:
        try:
            if len(event_list) == max_Read: #FRANK# limit event size. If violated, interrupt the entire method
                return(event_list,mass_list)
            if weighted == 0:
                                
                #FRANK# rfind finds the last occurance
                #FRANK# [a:b] extract everything b/w a and bth elem, including ath and excluding bth
                #FRANK# why isn't the cut applied to weight and mass list? #ISSUE#
                
                mass_list.append(float(x[x.rfind("\n")+1:-1]))
                to_cut = np.array(np.genfromtxt(x[:x.rfind("\n")].splitlines(), delimiter=","))
                event_list.append(to_cut[[x[0] > pt_cut for x in to_cut]])
            else:
                weight_list.append(float(x[x.rfind("\n")+1:]))
                mass_list.append(float(x[x.rfind("\n",0,x.rfind("\n")-1)+1:x.rfind("\n")+1]))
                to_cut = np.genfromtxt(x[:x.rfind("\n")].splitlines(),delimiter=",")
                event_list.append(to_cut[[x[0] > pt_cut for x in to_cut]])
        except:
            print('We failed to turn your CSV into an np array. Make sure you have the correct format. ')
            print( sys.exc_info()[0])
            return
    if weighted == 0:
        #print(mass_list[0])
        return(event_list,mass_list)
    else:
        return(event_list,mass_list,weight_list)

#FRANK# Converting event_list to event image
def return_image_list(event_list):
    image_list = []
    image_0 = np.zeros((width,height)) #Charged pt #FRANK# I think it's labeled wrong here. Would it matter?
    image_1 = np.zeros((width,height)) #Neutral pt
    image_2 = np.zeros((width,height)) #Charged multiplicity
    
    #FRANK# pt, eta, phi, m, id, isCharged \n
    #FRANK# 0   1    2    3  4   5
    for z in range(len(event_list)):
        image_0 = np.zeros((width,height));image_1 = np.zeros((width,height));image_2 = np.zeros((width,height))
        for x in range(len(event_list[z])):
            phi_index = math.floor(width*event_list[z][x,2]//(2*math.pi)+width//2)
            eta_index = math.floor(height*event_list[z][x,1]//10+height/2) #FRANK# // is integer divide
            eta_index = min(eta_index,height-1)
            eta_index = max(0,eta_index)
            phi_index = int(phi_index);eta_index = int(eta_index)
            if (event_list[z][x,5] == 0):  #FRANK# neutral
                image_0[phi_index,eta_index] = image_0[phi_index,eta_index] + event_list[z][x,0]
            elif (event_list[z][x,5] == 1):  #FRANK# charged
                image_1[phi_index,eta_index] = image_1[phi_index,eta_index] + event_list[z][x,0]
                image_2[phi_index,eta_index] = image_2[phi_index,eta_index] + 1
        image_0 = np.divide(image_0,np.sum(image_0))
        image_1 = np.divide(image_1,np.sum(image_1))
        image_2 = np.divide(image_2,np.sum(image_2))
        image_list.append(np.array([image_0,image_1,image_2]))
    return(image_list)

# This method loads event files to produce:
# event_list, mass_list, image_list, files_Read and weight_list
def load_events(event_type,debug = 0, max_Read = float("inf"), max_Files = float("inf"), weighted=0, \
                path = "/data1/users/jzlin/NLO/", contains = "philback", pt_cut = 0):
    print("Loading events for " + event_type)
    reading_event_list,reading_mass_list = [],[]
    reading_image_list = []
    reading_weight_list = []
    files_Read = 0
    print('list of files is'+ str(os.listdir(path)))
    for i in os.listdir(path):
        print('test: i is: '+ str(i))
        print('total path is: '+os.path.join(path,i))
        
        if (files_Read == max_Files):
            break
        if len(reading_event_list) >= max_Read:
            return(reading_event_list,reading_mass_list,reading_image_list,files_Read)
        if 'swp' in i:
            continue
        if os.path.isfile(os.path.join(path,i)) and (event_type+contains) in i:
            if debug==1:
                print(i)
                print(os.path.join(path,i))
            if weighted==0:
                print(i)
                temp_event_list,temp_mass_list = return_event_list(os.path.join(path,i),pt_cut=pt_cut)
            else:
                temp_event_list,temp_mass_list,temp_weight_list = return_event_list(os.path.join(path,i),
                                                                                    weighted=1,pt_cut=pt_cut)
                reading_weight_list = reading_weight_list = temp_weight_list
            temp_image_list = return_image_list(temp_event_list)
            if (len(temp_image_list) != len(temp_mass_list)):
                print("Something has gone wrong when reading the file")
                print(os.path.join(path,i))
            reading_event_list = reading_event_list + temp_event_list
            reading_mass_list = reading_mass_list + temp_mass_list
            reading_image_list = reading_image_list + temp_image_list
            files_Read = files_Read + 1
            print("Read " + str(files_Read) + " number of files\r")
    if weighted==0:
        return(reading_event_list,reading_mass_list,reading_image_list,files_Read)
    else:
        return(reading_event_list,reading_mass_list,reading_image_list,files_Read,reading_weight_list)

# This is not used. 
def return_fine_image_list(event_list, event_list_clustered, granularity, which_jet = 0):
    image_list = []
    image_0 = np.zeros((width,height)) #Charged pt
    image_1 = np.zeros((width,height)) #Neutral pt
    image_2 = np.zeros((width,height)) #Charged multiplicity

    for z in range(len(event_list)):
        image_0 = np.zeros((width,height))
        image_1 = np.zeros((width,height))
        image_2 = np.zeros((width,height))
        for x in range(len(event_list[z])):
            
            try:
                phi_index = (event_list[z][x,2]-event_list_clustered[z][which_jet].phi)
            except:
                print(z)
            #At this point, phi_index is just delta_phi, which could be anywhere from -2pi to 2pi
            if (phi_index % (2*math.pi) >= (width//2)*granularity) and (phi_index % (2*math.pi) <= 2*math.pi-(width//2)*granularity):
                continue
                #This gets rid of the delta phi's that are far away from the jet
            phi_index = phi_index % (2*math.pi)
            if phi_index > math.pi:
                 phi_index = phi_index - 2*math.pi   
            phi_index = int(math.floor(phi_index/granularity)) #should be good now
            if (phi_index > (width//2)) or (phi_index < -(width//2)):
                print(phi_index)
            phi_index = phi_index + (width//2)
            

            eta_index = int(math.floor((event_list[z][x,1]-event_list_clustered[z][which_jet].eta)/granularity) + height//2)
            if eta_index >= height:
                continue
            if eta_index < 0:
                continue
            
            #finally, lets fill
            if (event_list[z][x,5] == 0):
                image_0[phi_index,eta_index] = image_0[phi_index,eta_index] + event_list[z][x,0]
            elif (event_list[z][x,5] == 1):
                image_1[phi_index,eta_index] = image_1[phi_index,eta_index] + event_list[z][x,0]
                image_2[phi_index,eta_index] = image_2[phi_index,eta_index] + 1

        #Now, lets go through and normalise to 255
        image_0 = np.divide(image_0,np.sum(image_0))
        image_1 = np.divide(image_1,np.sum(image_1))
        image_2 = np.divide(image_2,np.sum(image_2))
        image_list.append(np.array([image_0,image_1,image_2]))
    return(image_list)

def cluster_event(event_list):
    event_list_clustered = []
    for x in range(len(event_list)):
        to_Cluster = np.array([event_list[x][:,0],event_list[x][:,1],event_list[x][:,2],event_list[x][:,3]])
        to_Cluster = np.swapaxes(to_Cluster,0,1)
        to_Cluster = np.core.records.fromarrays(to_Cluster.transpose(), 
                                             names='pT, eta, phi, mass',
                                             formats = 'f8, f8, f8,f8')
        sequence_Cluster = cluster(to_Cluster, R = 0.8,p = -1)
        jets_Cluster = sequence_Cluster.inclusive_jets()
        event_list_clustered.append(jets_Cluster)
    return(event_list_clustered)

def recluster_event(cluster_list):
    reclustered_list= []
    for i in range(len(cluster_list)):
        sequence_Cluster = cluster((cluster_list[i][0]), R=0.2,p=-1)
        jets_Cluster = sequence_Cluster.inclusive_jets()
        reclustered_list.append(jets_Cluster)
    return(reclustered_list)

def return_fine_image_list_reclustered(event_list, event_list_clustered, radius, which_jet = 0,verbose = False):
    image_list = []
    image_0 = np.zeros((width,height)) #Neutral pt
    image_1 = np.zeros((width,height)) #Charged pt
    image_2 = np.zeros((width,height)) #Charged multiplicity
    
    no_two = 0

    for z in range(len(event_list)):
        image_0 = np.zeros((width,height))
        image_1 = np.zeros((width,height))
        image_2 = np.zeros((width,height))
        
        if (len(event_list_clustered[z]) > 1):
            #First, let's find the direction of the second-hardest jet relative to the first-hardest subjet
            phi_dir = -(dphi(event_list_clustered[z][1].phi,event_list_clustered[z][0].phi))
            eta_dir = -(event_list_clustered[z][1].eta - event_list_clustered[z][0].eta)
            #Norm difference:
            norm_dir = np.linalg.norm([phi_dir,eta_dir])
            #This is now the y-hat direction. so we can actually find the unit vector:
            y_hat = np.divide([phi_dir,eta_dir],np.linalg.norm([phi_dir,eta_dir]))
            #and we can find the x_hat direction as well
            x_hat = np.array([y_hat[1],-y_hat[0]]) 
        else:
            no_two = no_two + 1
            #continue
            
        if verbose==True:
            print(x_hat,y_hat,norm_dir)
            
        
        for x in range(len(event_list[z])):
            if (len(event_list_clustered[z]) == 1):
                #In the case that the reclustering only found one hard jet (that seems kind of bad, but hey)
                #no_two = no_two+1
                new_coord = [dphi(event_list[z][x,2],event_list_clustered[z][0].phi),event_list[z][x,1]-event_list_clustered[z][0].eta]
                indxs = [math.floor(width*new_coord[0]/(radius*1.5))+width//2,math.floor(height*(new_coord[1])/(radius*1.5))+height//2]
            else:
                #Now, we want to express an incoming particle in this new basis:
                part_coord = [dphi(event_list[z][x,2],event_list_clustered[z][0].phi),event_list[z][x,1]-event_list_clustered[z][0].eta]
                new_coord = np.dot(np.array([x_hat,y_hat]),part_coord)
                #Now, we want to cast these new coordinates into our array
                indxs = [math.floor(width*new_coord[0]/(radius*1.5))+width//2,math.floor(height*(new_coord[1]+norm_dir/1.5)/(radius*1.5))+height//2]
                
            if indxs[0] >= width or indxs[1] >= height or indxs[0] <= 0 or indxs[1] <= 0:
                continue
            phi_index = int(indxs[0]); eta_index = int(indxs[1])
            #finally, lets fill
            if (event_list[z][x,5] == 0):
                image_0[phi_index,eta_index] = image_0[phi_index,eta_index] + event_list[z][x,0]
            elif (event_list[z][x,5] == 1):
                image_1[phi_index,eta_index] = image_1[phi_index,eta_index] + event_list[z][x,0]
                image_2[phi_index,eta_index] = image_2[phi_index,eta_index] + 1

        #Now, lets go through and normalise to 255
        if (np.sum(image_0) == 0 or np.sum(image_1) == 0 or np.sum(image_2) == 0):
            image_list.append(np.array([image_0,image_1,image_2]))
            continue
        image_0 = np.divide(image_0,np.sum(image_0))
        image_1 = np.divide(image_1,np.sum(image_1))
        image_2 = np.divide(image_2,np.sum(image_2))
        image_list.append(np.array([image_0,image_1,image_2]))
    print("no two " + str(no_two))
    return(image_list)

# Correct phi range
def fix_phi(phi):
    while phi > math.pi:
        phi = phi - 2*math.pi
    while phi < -math.pi:
        phi = phi + 2*math.pi
    return phi

# Returns the difference in phi between phi, and phi_center
# as a float between (-PI, PI)
def dphi(phi,phi_c):
    
    dphi_temp = phi - phi_c
    while dphi_temp > np.pi:
        dphi_temp = dphi_temp - 2*np.pi
    while dphi_temp < -np.pi:
        dphi_temp = dphi_temp + 2*np.pi
    return (dphi_temp)

# Rapidity

def y(p):
    return ((1/2)*math.log((p.e+p.pz)/(p.e-p.pz)))

def R(con1,con2):
    return (((con1.eta-con2.eta)**2+dphi(con1.phi,con2.phi)**2)**(1/2))

def R_y(con1,con2):
    return (((y(con1)-y(con2))**2+dphi(con1.phi,con2.phi)**2)**(1/2))

def N_2(jcon):
    #Takes jcon, jet constituents
    p_x_total = np.sum([con.px for con in jcon])
    p_y_total = np.sum([con.py for con in jcon])
    p_total = (p_x_total**2+p_y_total**2)**(1/2)
    
    v_1e2 = 0
    for i in range(len(jcon)):
        for j in range(i+1,len(jcon)):
            v_1e2 = v_1e2+ jcon[i].pt*jcon[j].pt*R(jcon[i],jcon[j])/(p_total**2)
    v_2e3 = 0
    for i in range(len(jcon)):
        for j in range(i+1,len(jcon)):
            for k in range(j+1,len(jcon)):
                v_2e3 = v_2e3 + jcon[i].pt*jcon[j].pt*jcon[j].pt*min(R(jcon[i],jcon[j])*R(jcon[i],jcon[k]),
                                                                     R(jcon[j],jcon[k])*R(jcon[i],jcon[j]),
                                                        R(jcon[i],jcon[k])*R(jcon[j],jcon[k]))/(p_total**3)
    return v_2e3/(v_1e2**2)

# Softdrop 

class myJet(object):
    def __init__(self,px,py,pz):
        self.px = px; self.py = py; self.pz = pz; self.pt = (px**2+py**2)**(1/2)
        self.phi = math.atan2(py,px); self.eta = -math.log(math.tan(math.atan2(self.pt,self.pz)/2));

class Node(object):
    def __init__(self,data):
        self.data = data
        self.children = []
    def add_child(self,obj):
        self.children.append(obj)

def softdrop(jcon,z=0.1,debug = 0):
    #Takes the constituents of a jet, and softdrops it.
    #First, we need to step through the jet and build the tree of clustering
    #Since we are reclustering the whole thing; just take R = 1; i.e. dont need to think about it
    def distance(con1,con2):
        return R(con1,con2)**2
    pseudojets = []
    nodes = []
    for con in jcon:
        x = Node([con,1])
        #pseudojets.append(x)
        nodes.append(x) #1 means its still a pseudojet; i.e. not been clustered
    def how_many_pseudo(nodes):
        how_many = 0
        for node in nodes:
            if node.data[1] == 1:
                how_many = how_many + 1
        return how_many
    if debug == 1:
        print("len(nodes) : " + str(len(nodes)))
        #print(nodes)
    rep = 0
    while how_many_pseudo(nodes) > 1:
        #print(how_many_pseudo(nodes))
        min_distance = float("inf")
        min_index = [0,0]
        for i in range(0,len(nodes)):
            if nodes[i].data[1] == 0: #Its already part of something else
                continue
            for j in range(i+1,len(nodes)):
                if nodes[j].data[1] == 0:
                    continue
                if distance(nodes[i].data[0],nodes[j].data[0]) < min_distance:
                    min_index[0] = i; min_index[1] = j; 
                    min_distance = distance(nodes[i].data[0],nodes[j].data[0])
        i = min_index[0];j=min_index[1];
        new_node = Node([myJet(nodes[i].data[0].px+nodes[j].data[0].px,
                           nodes[i].data[0].py+nodes[j].data[0].py,
                           nodes[i].data[0].pz+nodes[j].data[0].pz),1])
        new_node.add_child(nodes[i])
        new_node.add_child(nodes[j])
        nodes.append(new_node)
        nodes[i].data[1] = 0
        nodes[j].data[1] = 0
    
    #print(nodes)

    to_check = [] #nodes to check
    for i in range(len(nodes)):
        if nodes[i].data[1] == 1:
            to_check.append(nodes[i])
    softcon = []
    while len(to_check) > 0:
        #print(to_check)
        our_childs = to_check[0].children
        #print(our_childs)
        if len(our_childs) == 0:
            softcon.append(to_check[0].data[0])
            to_check.pop(0)
            continue
        if min(our_childs[0].data[0].pt,our_childs[1].data[0].pt)/  \
                        (our_childs[0].data[0].pt+our_childs[1].data[0].pt) > z:
            to_check.append(our_childs[0])
            to_check.append(our_childs[1])
        elif our_childs[0].data[0].pt > our_childs[1].data[0].pt:
            to_check.append(our_childs[0])
        else:
            to_check.append(our_childs[1])
        to_check.pop(0)
    return softcon

# Normalize and zero-center any image
def zero_center_and_normalize(background_images, signal_images):
    tmp_av = np.average(np.concatenate((background_images, signal_images)), axis=0)
    tmp_sd = np.std(np.concatenate((background_images, signal_images)), axis=0)
    for i in range(len(background_images)):
        background_images[i] = np.divide((background_images[i] - tmp_av), (tmp_sd+1e-5)) #perhaps add some r to temp_sd to suppress noise
    for i in range(len(signal_images)):
        signal_images[i] = np.divide((signal_images[i] - tmp_av), (tmp_sd+1e-5))#/tmp_sd
    return background_images, signal_images

In [ ]:
# Reading in Josh's files; background is a single large file (pT cut must be 1 here for Josh's sample)
# This read produces event_list (collection of raw vectors) and event images
background_event_list,background_mass_list,background_image_list,num_background_files = \
    load_events("actual", max_Files=num_background_files,path="/data1/users/jzlin/MLM/background_7413/",\
                contains="_actual",pt_cut=1)
num_background_files = 15693
signal_event_list,signal_mass_list,signal_image_list,num_signal_files = \
    load_events("actual", max_Read = len(background_event_list),path="/data1/users/jzlin/MLM/heavy_signal/",\
                contains="_signal",pt_cut=1)

# Check size of dataset
print(len(background_mass_list),len(signal_mass_list))

# Zero centering and normalizing
background_image_list, signal_image_list = zero_center_and_normalize(background_image_list,signal_image_list)
background_mass_window = np.logical_and(np.array(background_mass_list) > 115,np.array(background_mass_list) < 135)

In [ ]:
# Calculate weights
background_weights = actual_background_cross*35.9*1e15/(average_number_accepted*num_background_files)
signal_weights = actual_signal_cross*35.9*1e15/(signal_accepted*num_signal_files)

In [ ]:
# Cluster events_lists into jets. The results are named background/signal_event_list_clustered
background_event_list_clustered = cluster_event(background_event_list)
signal_event_list_clustered = cluster_event(signal_event_list)

# Reclustering the events (i.e. clustering within events)
background_reclustered = recluster_event(background_event_list_clustered)
signal_reclustered = recluster_event(signal_event_list_clustered)

In [ ]:
# Produce jet images, the zero-center and normalize
background_recluster_images = return_fine_image_list_reclustered(background_event_list,
                                                           background_reclustered,0.8)
signal_recluster_images = return_fine_image_list_reclustered(signal_event_list,
                                                           signal_reclustered,0.8)

background_recluster_images, signal_recluster_images = zero_center_and_normalize(signal_recluster_images)

In [ ]:
print(background_recluster_images[0].shape)

# SPLITTING DATA

In [ ]:
# Function that splits whole event dataset into test, train and evaluation sets.
# the *_cut outputs has a 115<mass<135 cut.
batch_size = 50
epochs = 100
rsplit = np.array([0.5,0.75])

def splitData(background_image_list,signal_image_list):
    try:
        input_shape = background_image_list[0].shape
    except AttributeError:
        input_shape = (0)
        
    b_split = np.split(background_image_list,(len(background_image_list)*rsplit).astype(int))
    s_split = np.split(signal_image_list,(len(signal_image_list)*rsplit).astype(int))
    bm_split = np.split(background_mass_list,(len(background_mass_list)*rsplit).astype(int))
    sm_split = np.split(signal_mass_list,(len(signal_mass_list)*rsplit).astype(int))
    
    x_train = np.concatenate((b_split[0],s_split[0]))
    y_train = np.array(np.concatenate((np.zeros(len(b_split[0])),np.ones(len(s_split[0])))))
    mass_train = np.concatenate((bm_split[0],sm_split[0]))
    
    x_val = np.concatenate((b_split[1],s_split[1]))
    y_val = np.array(np.concatenate((np.zeros(len(b_split[1])),np.ones(len(s_split[1])))))
    mass_val = np.concatenate((bm_split[1],sm_split[1]))
    
    x_test = np.concatenate((b_split[2],s_split[2]))
    y_test = np.array(np.concatenate((np.zeros(len(b_split[2])),np.ones(len(s_split[2])))))
    mass_test = np.concatenate((bm_split[2],sm_split[2]))
        
    
    x_train_cut = [] #Note that we *train* on this cut sample, but *test* on the whole thing I guess
    y_train_cut = []
    mass_train_cut = []
    for x in reversed(range(len(x_train))):
        if mass_train[x] < 135 and mass_train[x] > 115:
            x_train_cut.append(x_train[x])
            y_train_cut.append(y_train[x])
            mass_train_cut.append(mass_train[x])
    x_train_cut = np.array(x_train_cut)
    y_train_cut = np.array(y_train_cut)
    mass_train_cut = np.array(mass_train_cut)
    
    x_val_cut = [] 
    y_val_cut = []
    mass_val_cut = []
    for x in reversed(range(len(x_val))):
        if mass_val[x] < 135 and mass_val[x] > 115:
            x_val_cut.append(x_val[x])
            y_val_cut.append(y_val[x])
            mass_val_cut.append(mass_val[x])
    x_val_cut = np.array(x_val_cut)
    y_val_cut = np.array(y_val_cut)
    mass_val_cut = np.array(mass_val_cut)
    
    x_test_cut = [] 
    y_test_cut = []
    mass_test_cut = []
    for x in reversed(range(len(x_test))):
        if mass_test[x] < 135 and mass_test[x] > 115:
            x_test_cut.append(x_test[x])
            y_test_cut.append(y_test[x])
            mass_test_cut.append(mass_test[x])
    x_test_cut = np.array(x_test_cut)
    y_test_cut = np.array(y_test_cut)
    mass_test_cut = np.array(mass_test_cut)
    
    return(input_shape,
           x_train,y_train,mass_train,
           x_val,y_val,mass_val,
           x_test,y_test,mass_test,
           x_train_cut,y_train_cut,mass_train_cut,
           x_val_cut,y_val_cut,mass_val_cut,
           x_test_cut,y_test_cut,mass_test_cut,
           )

In [ ]:
# Split event images
input_shape, \
    x_train,y_train,mass_train, \
    x_val,y_val,mass_val, \
    x_test,y_test,mass_test, \
    x_train_cut,y_train_cut,mass_train_cut, \
    x_val_cut,y_val_cut,mass_val_cut, \
    x_test_cut,y_test_cut,mass_test_cut = splitData(background_image_list,signal_image_list)

In [ ]:
# Split jet images
input_shape_r, \
    x_train_r,y_train_r,mass_train_r, \
    x_val_r,y_val_r,mass_val_r, \
    x_test_r,y_test_r,mass_test_r, \
    x_train_cut_r,y_train_cut_r,mass_train_cut_r, \
    x_val_cut_r,y_val_cut_r,mass_val_cut_r, \
    x_test_cut_r,y_test_cut_r,mass_test_cut_r = splitData(background_recluster_images,signal_recluster_images)

In [ ]:
#FRANK print an event image
print(np.shape(x_train[0]))
plt.imshow(x_train_r[-1][0])
plt.show()

plt.imshow(x_train_r[-1][1])
plt.show()

plt.imshow(x_train_r[-1][2])
plt.show()

# ANALYSIS FUNCTIONS

In [ ]:
def generate_real_SIC(expect,predict,masses,quality=1,verbose=False):
    to_sort = np.flip(np.array(sorted(np.vstack((expect,predict.flatten(),masses)).transpose(), key=lambda x: x[1])),0)
    background_weights = actual_background_cross*35.9*1e15/(average_number_accepted*num_background_files)
    signal_weights = actual_signal_cross*35.9*1e15/(signal_accepted*num_signal_files)
    efficiency = []; signal_eff = [];
    total_signal = np.sum(to_sort[:,0])
    for i in range(int(0.05*len(to_sort)),len(to_sort),quality*10):
        background_mass_binned,bins = np.histogram(to_sort[:i+1,2][to_sort[:i+1,0]==0],bins=np.arange(50,197,7))
        signal_mass_binned,bins = np.histogram(to_sort[:i+1,2][to_sort[:i+1,0]==1],bins=np.arange(50,197,7))
        log_likelihood = 0; baseLL = 0; 
        signal_eff.append(np.sum(to_sort[:i+1,0])/total_signal)
        test_LL_vs_SS = []
        for signal_strength in np.arange(1,5,quality/1000):
            log_likelihood = 0;
            for k in range(len(bins)-1):
                expected = background_weights*background_mass_binned[k]+signal_weights*signal_strength*signal_mass_binned[k]
                observed = background_weights*background_mass_binned[k]+signal_weights*signal_mass_binned[k]
                if (expected <= 0):
                    pass
                else:
                    log_likelihood = log_likelihood + observed*math.log(expected) - expected
            if signal_strength == 1:
                baseLL = log_likelihood
            test_LL_vs_SS.append(log_likelihood)
            if log_likelihood < baseLL-1/2:
                efficiency.append(1/(signal_strength-1))
                break
            if signal_strength > 3:
                efficiency.append(1/2)
                break
    max_eff = np.max(efficiency)
    max_cut = to_sort[efficiency.index(max_eff),1]
    print("base efficiency : " + str(float(efficiency[-1])))
    efficiency = np.array(efficiency)/float(efficiency[-1])
    max_eff = np.max(efficiency)
    print("Max SI of " + str(max_eff) + " at cut " + str(max_cut))
    return(efficiency,signal_eff)

def find_highest_SIC(expect,predict,quality=100,verbose=False):
    to_sort = np.flip(np.array(sorted(np.vstack((expect,predict.flatten())).transpose(), key=lambda x: x[1])),0)
    total_signal = np.sum(to_sort[:,0])
    efficiency = []
    for i in range(int(0.05*len(to_sort)),len(to_sort)): # generate a int range scanning over 95% of all samples??
        signal_eff_temp = np.sum(to_sort[:i+1,0])/total_signal
        background_eff_temp = (i+1-np.sum(to_sort[:i+1,0]))/(len(to_sort)-total_signal)
        efficiency.append((signal_eff_temp)/((background_eff_temp)**(1/2)))
    max_eff = np.max(efficiency)
    return(max_eff)

def find_highest_SIC_binned(expect,predict,masses):
    bins = np.arange(50,197,7)
    efficiency = []
    def log_like(signal_strength,background_mass_list,signal_mass_list):
        log_likelihood = 0
        background_mass_binned,bins = np.histogram(background_mass_list,bins=np.arange(50,197,7))
        signal_mass_binned,bins = np.histogram(signal_mass_list,bins=np.arange(50,197,7))
        background_weights = actual_background_cross*35.9*1e15/(average_number_accepted*num_background_files)
        signal_weights = actual_signal_cross*35.9*1e15/(signal_accepted*num_signal_files)
        for i in range(len(bins)-1):
            expected = background_weights*background_mass_binned[i]+signal_weights*signal_strength*signal_mass_binned[i]
            observed = background_weights*background_mass_binned[i]+signal_weights*signal_mass_binned[i]
            if (expected <= 0):
                return float("inf")
            log_likelihood = log_likelihood + observed*math.log(expected) - expected
        return -log_likelihood
    
    sigmas = []
    for i in np.arange(0,0.9,0.05):
        #res = least_squares(lambda x : log_like(x,masses[np.logical_and(predict.flatten() >= i,expect == 0)],
        #                                        masses[np.logical_and(predict.flatten() >= i,expect == 1)]),x0=1)

        #i is the cut on the machine learning
        kept_back = masses[np.logical_and(predict.flatten() >= i,expect == 0)]
        kept_signal = masses[np.logical_and(predict.flatten() >= i,expect == 1)]
        j_array = []
        for j in np.arange(1,25,0.5):
            #print log_like(j,kept_back,kept_signal)
            if log_like(j,kept_back,kept_signal) > log_like(1,kept_back,kept_signal)+0.5:
                j_array.append(j-1)
                break
            if j >20:
                j_array.append(20)
                break
        print(j_array,i)
        sigmas.append(1/np.min(j_array))
        
    max_eff = np.max(sigmas)
    return(max_eff)

def generateSIC(expect,predict,quality=100,verbose=False):
    to_sort = np.flip(np.array(sorted(np.vstack((expect,predict.flatten())).transpose(), key=lambda x: x[1])),0)
    total_signal = np.sum(to_sort[:,0])
    efficiency = []; signal_eff = []
    for i in range(int(0.05*len(to_sort)),len(to_sort)):
        signal_eff_temp = np.sum(to_sort[:i+1,0])/total_signal
        background_eff_temp = (i+1-np.sum(to_sort[:i+1,0]))/(len(to_sort)-total_signal)
        signal_eff.append(signal_eff_temp)
        efficiency.append((signal_eff_temp)/((background_eff_temp)**(1/2)))
    max_eff = np.max(efficiency)
    max_cut = to_sort[efficiency.index(max_eff),1]
    print("Max SI of " + str(max_eff) + " at cut " + str(max_cut))
    return(efficiency,signal_eff)

def log_like(signal_strength,background_mass_list,signal_mass_list):
    log_likelihood = 0
    background_mass_binned,bins = np.histogram(background_mass_list,bins=np.arange(50,197,7))
    signal_mass_binned,bins = np.histogram(signal_mass_list,bins=np.arange(50,197,7))
    background_weights = actual_background_cross*35.9*1e15/(average_number_accepted*num_background_files)
    signal_weights = 3.15*actual_signal_cross*35.9*1e15/(signal_accepted*num_signal_files)
    for i in range(len(bins)-1):
        expected = background_weights*background_mass_binned[i]+signal_weights*signal_strength*signal_mass_binned[i]
        observed = background_weights*background_mass_binned[i]+signal_weights*signal_mass_binned[i]
        if (expected <= 0):
            return float("inf")
        log_likelihood = log_likelihood + observed*math.log(expected) - expected
    return -log_likelihood

def log_like(signal_strength):
    log_likelihood = 0
    for i in range(len(bins)-1):
        expected = background_weights*background_mass_binned[i]+signal_weights*signal_strength*signal_mass_binned[i]
        observed = background_weights*background_mass_binned[i]+signal_weights*signal_mass_binned[i]
        #print(expected)
        log_likelihood = log_likelihood + observed*math.log(expected) - expected
    return log_likelihood

# Neural network utilities

In [ ]:
#FRANK# Returns a function that generates a padding layer
#FRANK# x is the detector image, of following format:
#FRANK# [index, image#(charged and neutral pt and multiplicity), 40, 40]
def return_pad_me(padding):
    def pad_me(x):
        #FRANK# x[:,:,:y,:] slice x off from y at the given axis.
        return(tf.concat((x,x[:,:,:padding,:]),2))
    return(pad_me)

def pad_out(padding,input_shape):
    return input_shape

class gen_call(keras.callbacks.Callback):
    
    def __init__(self, test_data):
        self.x, self.y = test_data
    
    def on_train_begin(self,logs={}):
        self.highest_SIC_train = []
        self.highest_SIC_test = []
        
    def on_epoch_end(self,epoch,logs={}):
        y_pred = self.model.predict(self.x)
        self.highest_SIC_test.append(find_highest_SIC(self.y,y_pred))
        print(str(self.highest_SIC_test[-1]) + " is how good")

def show_outputs(output):
    #Assumes the output is in shape like (32,41,36)
    
    fig = plt.figure(figsize=(8,6))
    
    for i in range(1,1+output.shape[0]):
        fig.add_subplot(4,output.shape[0]/4,i)
        plt.imshow(10*output[i-1,:,:])
        plt.axis('off')
    #plt.axis('off')
    plt.show()

In [ ]:
#FRANK# this method is the exact replica of find_highest_SIC() but operates on tensors.
#FRANK# for some reason, pred has different format from expect. 
def highest_SIC_metric(y_true,y_pred):
    print('highest_SIC_metric() is called')
    y_true = tf.keras.backend.flatten(y_true) #FRANK# flattens to 1D
    y_pred = tf.keras.backend.flatten(y_pred) #FRANK# flattens to 1D
    
    #stacked = tf.transpose(tf.stack((expect,predict)))
    #to_sort = tf.reverse(sorted(stacked, key=lambda x: x[1]),0) #FRANK# sorted is wrong!!
    total_sample = tf.cast(tf.size(y_pred), tf.float32)  # count total nums, then cast to float32 to avoid issue
    total_signal = tf.cast(tf.reduce_sum(y_true), tf.float32)  # summing across all predicted (where sigs are 1's and bkgs are 0's) to get total num of signal. 
    total_background = tf.cast(tf.subtract(total_sample, total_signal), tf.float32)  # subtracting signal countss from total size to get background counts
    
    
    # original mechanism:
    # 1. sort by ML score
    # 2. sum all actuals before an index
    # 3. count all 0's before an index 
    
    sorted_indices = tf.argsort(y_pred,axis=-1,direction='ASCENDING') # tf.argsort: Returns the indices of a tensor that give its sorted order along an axis.
    sorted_sigs = tf.gather(y_pred,sorted_indices)
    # return the indices of prediction in ascending order. By reading these indices, you can access corresponding expected y's.
    ones = tf.fill(tf.shape(sorted_sigs), 1.0)
    sorted_bkgs = tf.subtract(ones, sorted_sigs)
    # make a sorted tensor where 1's are bkgs and 0's are sigs
    
    sig_cum_sums = tf.cast(tf.cumsum(sorted_sigs), tf.float32) # return the integrated signal number from 0 to each index
    bkg_cum_sums = tf.cast(tf.cumsum(sorted_bkgs), tf.float32) # return the integrated bkg number from 0 to each index

    sig_effs = tf.divide(sig_cum_sums, total_signal)
    bkg_effs = tf.divide(bkg_cum_sums, total_background) #FRANK# total_background might be 0, causing bkg_effs=inf
    effs = tf.divide(sig_effs, tf.sqrt(bkg_effs))
        
    return tf.reduce_max(effs)

# Models

In [ ]:
# --------------------------------------------------------------------------------------------
# Event image model
# --------------------------------------------------------------------------------------------
event_image_cnn = Sequential()
event_image_cnn.add(Lambda(return_pad_me(4),
                 input_shape=input_shape))
event_image_cnn.add(Conv2D(32, kernel_size=(5, 5), strides=(1, 1),
                 activation='relu',
                data_format='channels_first'))
event_image_cnn.add(Lambda(return_pad_me(1),
                 input_shape=input_shape))
event_image_cnn.add(MaxPooling2D(pool_size=(2, 2), strides=(2, 2),data_format='channels_first'))
event_image_cnn.add(Lambda(return_pad_me(4),
                 input_shape=input_shape))
event_image_cnn.add(Conv2D(64, (5, 5), 
                 activation='relu',
                 data_format='channels_first'))
event_image_cnn.add(Lambda(return_pad_me(1),
                 input_shape=input_shape))
event_image_cnn.add(MaxPooling2D(pool_size=(2, 2),data_format='channels_first'))
event_image_cnn.add(Flatten())
event_image_cnn.add(Dense(1000, activation='relu'))
event_image_cnn.add(Dense(1, activation='sigmoid'))
event_image_cnn.summary()

#model_opt = keras.optimizers.Adadelta(lr=2.0, rho=0.95, epsilon=None, decay=0.0)
model_opt = keras.optimizers.Adadelta() #FRANK# decrease learning rate

event_image_cnn.compile(loss=keras.losses.binary_crossentropy,
              optimizer=model_opt,
              metrics=['accuracy', highest_SIC_metric])



# --------------------------------------------------------------------------------------------
# Jet image model
# --------------------------------------------------------------------------------------------
jet_image_cnn = Sequential()
jet_image_cnn.add(Conv2D(32, kernel_size=(5, 5), strides=(1, 1),
                 activation='relu',
                data_format='channels_first',input_shape=input_shape))
#jet_image_cnn.add(Dropout(0.5))
jet_image_cnn.add(MaxPooling2D(pool_size=(2, 2), strides=(2, 2),data_format='channels_first'))
jet_image_cnn.add(Conv2D(64, (5, 5), activation='relu',data_format='channels_first'))
#jet_image_cnn.add(Dropout(0.5))
jet_image_cnn.add(MaxPooling2D(pool_size=(2, 2),data_format='channels_first'))
jet_image_cnn.add(Flatten())
jet_image_cnn.add(Dense(1000, activation='relu'))
#jet_image_cnn.add(Dropout(0.2))
jet_image_cnn.add(Dense(1, activation='sigmoid'))
#jet_image_cnn.summary()

model_opt = keras.optimizers.Adadelta()

jet_image_cnn.compile(loss=keras.losses.binary_crossentropy,
              optimizer=model_opt,
              metrics=['accuracy', 'highest_SIC_test'])



# --------------------------------------------------------------------------------------------
# Combined model
# --------------------------------------------------------------------------------------------
#FRANK# input shape is calculated during splitting:
#FRANK# splitData(background_image_list,signal_image_list)
#FRANK# how is jet image taken?
event_image_branch = Sequential()

#FRANK# Lambda is from Keras.
event_image_branch.add(Lambda(return_pad_me(5), #FRANK# tf.concat((x,x[:,:,:5,:]),2). Concatenate means pasting together.
                            #FRANK# this layer slices off an edge of the tensor and paste it on the other side.
                 input_shape=input_shape))
event_image_branch.add(Conv2D(32, kernel_size=(5, 5), strides=(1, 1),
                 activation='relu',
                data_format='channels_first'))
event_image_branch.add(Lambda(return_pad_me(2)))
event_image_branch.add(MaxPooling2D(pool_size=(2, 2), strides=(2, 2)))
event_image_branch.add(Lambda(return_pad_me(5),
                 input_shape=input_shape))
event_image_branch.add(Conv2D(64, (5, 5), activation='relu'))
#event_image_branch.add(Dropout(0.5))
event_image_branch.add(Lambda(return_pad_me(2)))
event_image_branch.add(MaxPooling2D(pool_size=(2, 2)))
event_image_branch.add(Flatten())
event_image_branch.add(Dense(300,activation='relu'))
#event_image_branch.add(Dropout(0.5))

jet_image_branch = Sequential()
jet_image_branch.add(Conv2D(32, kernel_size=(5, 5), strides=(1, 1),
                 activation='relu',
                      kernel_initializer='random_uniform',input_shape=input_shape_r, data_format="channels_first")) #FRANK# problemmatic. see debug note #1
#jet_image_branch.add(Dropout(0.5))
jet_image_branch.add(MaxPooling2D(pool_size=(2, 2), strides=(2, 2)))
jet_image_branch.add(Conv2D(64, (5, 5), activation='relu'))
#jet_image_branch.add(Dropout(0.5))
jet_image_branch.add(MaxPooling2D(pool_size=(2, 2)))
jet_image_branch.add(Flatten())
jet_image_branch.add(Dense(300, activation='relu'))
#jet_image_branch.add(Dropout(0.5))



#FRANK# Merge layer is no longer supported.  
#combined_model = Sequential()
#combined_model.add(Merge([event_image_branch, jet_image_branch], mode = 'concat'))
#combined_model.add(Dense(1,activation='sigmoid'))

#FRANK# these are not models, but tensors. Model.input or Model.output are tensor pointers I think.
combined_model_tensor = Concatenate(axis=-1)([event_image_branch.output, jet_image_branch.output])   
combined_model_tensor = Dense(1,activation='sigmoid')(combined_model_tensor)
#FRANK# what you need to do is merging output of 300 and 300 into one.

#combined_model.summary()
combined_model = Model([event_image_branch.input,jet_image_branch.input], combined_model_tensor)
combined_model.compile(loss=keras.losses.binary_crossentropy,
              optimizer=keras.optimizers.Adadelta(),
              metrics=['accuracy', highest_SIC_metric])

# Visualizing event image model
plot_model(combined_model, show_shapes=True)

In [ ]:
#FRANK# Sample training call testing metric
epochs = 100
print(y_train)
with tf.device('/device:XLA_GPU:3'):
    history_event_only = event_image_cnn.fit(x_train, y_train,
        batch_size=1024,#batch_size,
        epochs=epochs,
        validation_data=(x_val,y_val),
        verbose=1, shuffle=True, callbacks=[EarlyStopping(monitor='highest_SIC_test', patience=2), gen_call((x_val_cut,y_val_cut))])

#FRANK# for event image SIC: predicting
with tf.device('/device:XLA_GPU:3'):
    y_pred_test = model.predict(x_test)
    
print(y_pred_test)

In [ ]:
#FRANK# testing metric
print(np.array(y_pred.flatten()))
print(np.array(y_test))
print(highest_SIC_metric(y_pred,y_test))

In [ ]:
#FRANK# training history plots
def loss_history_plot(training_history):
    print(training_history.history.keys())
    plt.plot(training_history.history['loss'])
    plt.title('model accuracy')
    plt.ylabel('loss')
    plt.xlabel('epoch')
    plt.legend(['train', 'test'], loc='upper left')
    plt.show()


# Evaluation

In [ ]:
# Print groomed mass histogram
plt.rcParams['figure.figsize'] = [7.5,6]
background_weight = actual_background_cross*35.9*1e15/(average_number_accepted*num_background_files)
signal_weight = actual_signal_cross*35.9*1e15/(signal_accepted*num_signal_files)
background_weights = np.full(21, background_weight, dtype=None, order='C')
signal_weights = np.full(21, signal_weight, dtype=None, order='C')

hist_bkg, bin_edges_bkg = np.histogram(background_mass_list, bins=21, range = (50, 197))
bin_centers_bkg = (bin_edges_bkg[:-1] + bin_edges_bkg[1:]) / 2
hist_sig, bin_edges_bkg = np.histogram(signal_mass_list, bins=21, range = (50, 197))

plt.step(bin_centers_bkg, hist_bkg*background_weight, label='bkgd')
plt.step(bin_centers_bkg, hist_sig*signal_weight, label='sig')
plt.legend()
plt.ylim(top=10000)
plt.ylim(bottom=0)
plt.xlim(right=197)  # adjust the right leaving left unchanged
plt.xlim(left=50) 
plt.show()
#log_like(1,background_mass_list=background_mass_list,signal_mass_list=signal_mass_list)

# Calculating beta3
background_beta3 = find_new_var_beta_3(background_reclustered,background_event_list_clustered,pt_cut = 1)
signal_beta3 = find_new_var_beta_3(signal_reclustered,signal_event_list_clustered,pt_cut = 1)

# Beta-3 histogram

plt.rcParams['figure.figsize'] = [5, 4]
beta3_min = min([min(background_beta3),min(signal_beta3)])
beta3_max = max([max(background_beta3),max(signal_beta3)])
print('beta3 range is '+str((beta3_min, beta3_max)))
plt.hist(background_beta3, label='bkgd', bins = 40, range = (beta3_min, 70), stacked=True, density=True,histtype ='step')
plt.hist(signal_beta3, label='sig', bins = 40, range = (beta3_min, 70), stacked=True, density=True,histtype ='step')
plt.xlabel("Beta-3")
plt.ylabel("Normalized Shape")
plt.legend()
plt.show()

plt.hist(background_mass_list, label='bkgd', bins = 21, range = (50, 197), stacked=True, density=True,histtype ='step')
plt.hist(signal_mass_list, label='sig', bins = 21, range = (50, 197), stacked=True, density=True,histtype ='step')
plt.xlabel("Trimmed Mass")
plt.ylabel("Normalized Shape")
plt.legend()
plt.show()

# Beta-3 plots
plot_simple_ROC_SIC(signal_beta3, background_beta3, step = 0.001):

In [ ]:
# Split the predicted results into sample and background using known answers.
# Here we assume all backgrounds are put before signals in all datasets. 
#TODO# Check before use!

def split_sig_bkgd(predicted, expected)
    split_location = int(np.argwhere(expected==1.)[0])
    #FRANK# signal predictions with event image model
    sig = predicted[split_location:]
    bkgd = predicted[:split_location]

    return sig, bkgd

In [ ]:
#FRANK# calculating true positive rates and false positive rates. 
#FRANK# TPR = TP/P
#FRANK# FPR = FP/N
#FRANK# note that bkgds have lower beta in general, so the cut should be 
#FRANK# beta3 > threshold.
#FRANK# 'where' usage referred from 
#FRANK# https://stackoverflow.com/questions/12995937/count-all-values-in-a-matrix-greater-than-a-value

def simple_ROC(sig, bkgd, step = 1):
    thres_min = min([min(sig),min(bkgd)])
    thres_max = max([max(sig),max(bkgd)])
    search_range=(thres_min, thres_max)
    sig_count = len(sig)
    bkgd_count = len(bkgd)
    sig_rates = []
    bkgd_rates = []
    thres_list = np.arange(search_range[0], search_range[1], step)
    for thres in thres_list:
        sig_selected = np.greater(sig, thres)
        bkgd_selected = np.greater(bkgd, thres)
        sig_rates.append(np.sum(sig_selected)/sig_count)
        bkgd_rates.append(np.sum(bkgd_selected)/bkgd_count)
    return sig_rates, bkgd_rates, thres_list

def plot_simple_ROC_SIC(sig, bkgd, step = 0.001):
    
    # Calculate simple ROC curve
    sig_rates, bkgd_rates, thres_list = simple_ROC(sig, bkgd, step)
    

    # Plotting ROC
    plt.rcParams['figure.figsize'] = [6, 6]
    plt.plot(sig_rates, bkgd_rates,label='bkgd',linewidth=1)
    plt.legend()
    plt.xlabel("signal rate")
    plt.ylabel("background rate")
    plt.show()

    # Plotting SIC (ε/√ε_b) 
    event_significances = sig_rates/np.sqrt(bkgd_rates)
    plt.rcParams['figure.figsize'] = [5, 4]
    plt.plot(sig_rates,event_significances,label='beta3',linewidth=1)
    plt.legend()
    plt.xlabel("signal rate")
    plt.ylabel("significance improvement")
    plt.ylim(top=2.5)
    plt.ylim(bottom=1)
    plt.xlim(right=1)  # adjust the right leaving left unchanged
    plt.xlim(left=0.1) 
    plt.show()

In [ ]:
#Here evaluate ML performance by e.g.
#r_combine_y,r_combine_x = generate_real_SIC(y_test,new_model_combine.predict([x_test,x_test_r]),mass_test,quality=1)

r_full_y,r_full_x = generateSIC(y_test,predicted, quality=100)
#r_fine_y,r_fine_x = generate_real_SIC(y_test,model_fine.predict(x_test_r),mass_test,quality=1)

In [ ]:
plt.figure(figsize=(10,7))
plt.title("Binned Likelihood Significance Improvements for various Architectures")
#plt.plot(r_combine_x,r_combine_y,color="red",label="Full CNN Architecture",linewidth=1)
plt.plot(r_full_x,r_full_y,color="red",alpha=0.6,label="Event image only",linewidth=1,linestyle="dashed")
#plt.plot(r_fine_x,r_fine_y,color="red",alpha=0.6,label="Jet image only",linewidth=1,linestyle="dotted")
#plt.plot(beta_x,beta_y,color="blue",alpha=0.6,label=r"$\beta_3$",linewidth=1)#,linestyle="dashed")
#plt.plot(three_x,three_y,color="blue",alpha=0.6,label=r"$Rb_2$",linewidth=1,linestyle="dotted")
#plt.plot(x_02,y_02,color="gray",label="Jet image, no neutral layer",linewidth=1)
#plt.plot(vars_y,vars_x,color="blue",label=r"$\beta_3 + Rb_2$",linewidth=1)
plt.xlim(0.1,1)
plt.ylim(1,2.3)
plt.legend()
plt.xlabel("Signal Efficiency")
plt.ylabel("Significance Improvement")
plt.savefig('all_SIC.eps')
plt.show()

In [ ]:
plt.figure(figsize=(9,7))
plt.title("Binned Likelihood Significance Improvements for various Architectures")
#plt.plot(r_combine_x,r_combine_y,color="red",label="Full CNN Architecture",linewidth=1)
#plt.plot(r_full_x,r_full_y,color="red",alpha=0.6,label="Event image only",linewidth=1,linestyle="dashed")
#plt.plot(r_fine_x,r_fine_y,color="red",alpha=0.6,label="Jet image only",linewidth=1,linestyle="dotted")
#FRANK# event image SIC
plt.plot(r_full_x,r_full_y,color="red",alpha=0.6,label="Event image only",linewidth=1,linestyle="dashed")
plt.plot(sig_rates,significances,label='Beta3',linewidth=1)
#FRANK# beta-3 SIC
#plt.plot(beta_x,beta_y,color="blue",alpha=0.6,label=r"$\beta_3$",linewidth=1)#,linestyle="dashed")
#plt.plot(three_x,three_y,color="blue",alpha=0.6,label=r"$Rb_2$",linewidth=1,linestyle="dotted")
#plt.plot(x_02,y_02,color="gray",label="Jet image, no neutral layer",linewidth=1)
#plt.plot(vars_y,vars_x,color="blue",label=r"$\beta_3 + Rb_2$",linewidth=1)
plt.xlim(0.1,1)
plt.ylim(1,2.5)
plt.legend()
plt.xlabel("Signal Efficiency")
plt.ylabel("Significance Improvement")
plt.savefig('all_SIC.eps')
plt.show()